In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print("water")

water


In [21]:
dengue = pd.read_csv('DENGUE FEVER.csv')
dengue.head()

Unnamed: 0  year  week  month   adm3_pcode              adm3_en       tave  \
0           0  2008     1      1  PH137401000  City of Mandaluyong  25.845000   
1           1  2008     1      1  PH137503000      City of Navotas  25.436667   
2           2  2008     1      1  PH137603000   City of Muntinlupa  25.489901   
3           3  2008     1     12  PH137401000  City of Mandaluyong  26.446667   
4           4  2008     1     12  PH137503000      City of Navotas  26.156667   

        tmin       tmax  heat_index  ...  pharmacy_nearest  pop_count_total  \
0  23.536667  29.196667   27.045000  ...               NaN     11544.901089   
1  23.408333  28.186667   26.361667  ...               NaN     19420.567766   
2  23.190525  28.796780   26.600326  ...               NaN     49608.985460   
3  24.630000  28.930000   28.196667  ...               NaN     11544.901089   
4  24.513333  28.403333   27.556667  ...               NaN     19420.567766   

   pop_density_mean  clinic_count  dentist_count  doctors_count  \
0      31300.004557           0.0            0.0            0.0   
1      42940.326172           0.0            0.0            0.0   
2      10734.906445           0.0            0.0            0.0   
3      31300.004557           0.0            0.0            0.0   
4      42940.326172           0.0            0.0            0.0   

   hospital_count  optician_count  pharmacy_count  case_total  
0             0.0             0.0             0.0         NaN  
1             0.0             0.0             0.0         NaN  
2             0.0             0.0             0.0         NaN  
3             0.0             0.0             0.0         3.0  
4             0.0             0.0             0.0        16.0  

[5 rows x 30 columns]

In [22]:
print(dengue.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2811 non-null   int64  
 1   year              2811 non-null   int64  
 2   week              2811 non-null   int64  
 3   month             2811 non-null   int64  
 4   adm3_pcode        2811 non-null   object 
 5   adm3_en           2811 non-null   object 
 6   tave              2811 non-null   float64
 7   tmin              2811 non-null   float64
 8   tmax              2811 non-null   float64
 9   heat_index        2811 non-null   float64
 10  pr                2811 non-null   float64
 11  wind_speed        2811 non-null   float64
 12  rh                2811 non-null   float64
 13  solar_rad         2811 non-null   float64
 14  uv_rad            2811 non-null   float64
 15  clinic_nearest    1686 non-null   float64
 16  dentist_nearest   1686 non-null   float64


In [23]:
dengue_cleaned = dengue[dengue['year'] > 2014]
dengue_cleaned = dengue_cleaned.drop(['Unnamed: 0'], axis = 1)
dengue_cleaned.fillna(0, inplace = True)
dengue_cleaned = dengue_cleaned.apply(lambda x: x.map('{:.2f}'.format) if x.dtype == 'float64' else x)
adm3_pcode_mapping = {
    'PH137401000': 1,
    'PH137503000': 2,
    'PH137603000': 3
}
dengue_cleaned['adm3_pcode'] = dengue_cleaned['adm3_pcode'].map(adm3_pcode_mapping)

In [24]:
print(dengue_cleaned.info())

dengue_cleaned.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 1311 to 2810
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              1500 non-null   int64 
 1   week              1500 non-null   int64 
 2   month             1500 non-null   int64 
 3   adm3_pcode        1500 non-null   int64 
 4   adm3_en           1500 non-null   object
 5   tave              1500 non-null   object
 6   tmin              1500 non-null   object
 7   tmax              1500 non-null   object
 8   heat_index        1500 non-null   object
 9   pr                1500 non-null   object
 10  wind_speed        1500 non-null   object
 11  rh                1500 non-null   object
 12  solar_rad         1500 non-null   object
 13  uv_rad            1500 non-null   object
 14  clinic_nearest    1500 non-null   object
 15  dentist_nearest   1500 non-null   object
 16  doctors_nearest   1500 non-null   object
 17  hospital_ne

year  week  month  adm3_pcode              adm3_en   tave   tmin   tmax  \
1311  2015     1      1           1  City of Mandaluyong  24.90  23.05  27.55   
1312  2015     1      1           2      City of Navotas  24.67  22.73  27.29   
1313  2015     1      1           3   City of Muntinlupa  24.63  22.76  27.25   
1314  2015     2      1           1  City of Mandaluyong  25.38  23.15  28.33   
1315  2015     2      1           2      City of Navotas  25.27  23.12  28.03   

     heat_index    pr  ... pharmacy_nearest pop_count_total pop_density_mean  \
1311      25.77  2.20  ...           153.05        13551.19         35729.54   
1312      25.38  0.00  ...           949.48        20156.26         44536.11   
1313      25.46  6.40  ...           300.98        51784.80         11364.11   
1314      26.46  0.00  ...           153.05        13551.19         35729.54   
1315      26.20  0.00  ...           949.48        20156.26         44536.11   

     clinic_count dentist_count doctors_count hospital_count optician_count  \
1311         0.00          4.00          0.00           8.00           4.00   
1312         0.00          0.00          0.00           0.00           0.00   
1313         0.00         12.00          4.00          24.00           4.00   
1314         0.00          7.00          0.00          14.00           7.00   
1315         0.00          0.00          0.00           0.00           0.00   

     pharmacy_count case_total  
1311          72.00       0.00  
1312           4.00       0.00  
1313          72.00       0.00  
1314         126.00      10.00  
1315           7.00       9.00  

[5 rows x 29 columns]

In [25]:
dengue_cleaned.to_csv('Cleaned_Dengue_Data.csv', index=False)

In [26]:
leptospirosis = pd.read_csv('LEPTOSPIROSIS.csv')
leptospirosis.head()
print(leptospirosis.info())

leptospirosis_cleaned = leptospirosis[leptospirosis['year'] > 2014]
leptospirosis_cleaned = leptospirosis_cleaned.drop(['Unnamed: 0'], axis = 1)
leptospirosis_cleaned.fillna(0, inplace = True)
leptospirosis_cleaned = leptospirosis_cleaned.apply(lambda x: x.map('{:.2f}'.format) if x.dtype == 'float64' else x)
adm3_pcode_mapping = {
    'PH137401000': 1,
    'PH137503000': 2,
    'PH137603000': 3
}
leptospirosis_cleaned['adm3_pcode'] = leptospirosis_cleaned['adm3_pcode'].map(adm3_pcode_mapping)

print(leptospirosis_cleaned.info())
leptospirosis_cleaned.head()

dengue_cleaned.to_csv('Cleaned_Leptospirosis_Data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2811 non-null   int64  
 1   year              2811 non-null   int64  
 2   week              2811 non-null   int64  
 3   month             2811 non-null   int64  
 4   adm3_pcode        2811 non-null   object 
 5   adm3_en           2811 non-null   object 
 6   tave              2811 non-null   float64
 7   tmin              2811 non-null   float64
 8   tmax              2811 non-null   float64
 9   heat_index        2811 non-null   float64
 10  pr                2811 non-null   float64
 11  wind_speed        2811 non-null   float64
 12  rh                2811 non-null   float64
 13  solar_rad         2811 non-null   float64
 14  uv_rad            2811 non-null   float64
 15  clinic_nearest    1686 non-null   float64
 16  dentist_nearest   1686 non-null   float64


In [29]:
typoid = pd.read_csv('TYPHOID FEVER.csv')
typoid.head()
print(typoid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811 entries, 0 to 2810
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2811 non-null   int64  
 1   year              2811 non-null   int64  
 2   week              2811 non-null   int64  
 3   month             2811 non-null   int64  
 4   adm3_pcode        2811 non-null   object 
 5   adm3_en           2811 non-null   object 
 6   tave              2811 non-null   float64
 7   tmin              2811 non-null   float64
 8   tmax              2811 non-null   float64
 9   heat_index        2811 non-null   float64
 10  pr                2811 non-null   float64
 11  wind_speed        2811 non-null   float64
 12  rh                2811 non-null   float64
 13  solar_rad         2811 non-null   float64
 14  uv_rad            2811 non-null   float64
 15  clinic_nearest    1686 non-null   float64
 16  dentist_nearest   1686 non-null   float64


In [30]:

typoid_cleaned = typoid[typoid['year'] > 2014]
typoid_cleaned = typoid_cleaned.drop(['Unnamed: 0'], axis = 1)
typoid_cleaned.fillna(0, inplace = True)
typoid_cleaned = typoid_cleaned.apply(lambda x: x.map('{:.2f}'.format) if x.dtype == 'float64' else x)
adm3_pcode_mapping = {
    'PH137401000': 1,
    'PH137503000': 2,
    'PH137603000': 3
}
typoid_cleaned['adm3_pcode'] = typoid_cleaned['adm3_pcode'].map(adm3_pcode_mapping)

print(typoid_cleaned.info())
typoid_cleaned.head()

dengue_cleaned.to_csv('Cleaned_Typoid_Data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 1311 to 2810
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   year              1500 non-null   int64 
 1   week              1500 non-null   int64 
 2   month             1500 non-null   int64 
 3   adm3_pcode        1500 non-null   int64 
 4   adm3_en           1500 non-null   object
 5   tave              1500 non-null   object
 6   tmin              1500 non-null   object
 7   tmax              1500 non-null   object
 8   heat_index        1500 non-null   object
 9   pr                1500 non-null   object
 10  wind_speed        1500 non-null   object
 11  rh                1500 non-null   object
 12  solar_rad         1500 non-null   object
 13  uv_rad            1500 non-null   object
 14  clinic_nearest    1500 non-null   object
 15  dentist_nearest   1500 non-null   object
 16  doctors_nearest   1500 non-null   object
 17  hospital_ne